In [2]:
import pandas as pd

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
def DetectWords(df_func):
    df_func['tokenize'] = df_func['text'].apply(Tokenize)
    df_func['tokenized_text'] = df_func['tokenize'].apply(lambda tokens:' '.join(tokens).lower())
    df_cleaned_func = df_func.loc[:,['tokenized_text','label']]
    X_func = df_cleaned_func['tokenized_text']
    vectorizer_func = TfidfVectorizer()
    X_cleaned_func = vectorizer_func.fit_transform(X_func)
    for feature in vectorizer_func.get_feature_names_out():
        print(vectorizer_func.get_feature_names_out())
    return X_cleaned_func

In [5]:
df_fake_kaggle = pd.read_csv('Fake_Kaggle/train.csv')
df_fake_kaggle = df_fake_kaggle[(df_fake_kaggle['class'] == 'Fake') | (df_fake_kaggle['class'] == 'Real')]
df_fake_kaggle['class'] = df_fake_kaggle['class'].apply(lambda x : 0 if x == 'Real' else 1)
df_val_fake_kaggle = pd.DataFrame(pd.read_csv('Fake_Kaggle/test.csv').text,columns=['text'])

In [6]:
def GetFileTest(file_name):
    df = pd.read_csv(file_name)
    df = df.loc[:,['text','label']]
    df.dropna(inplace=True)
    df['label'] = df['label'].apply(lambda x : 0 if x == 1 else 1)
    return df

df_train_fake_news_kaggle = GetFileTest('Fake_News_Kaggle/train.csv')
df_val_fake_news_kaggle = pd.read_csv('Fake_News_Kaggle/test.csv').loc[:,['text']]
df_val_fake_news_kaggle.dropna(inplace=True)

In [7]:
def GetMappingLiarDataset(files_name):
    dfs = []
    for file_name in files_name:
        df_liar = pd.read_csv(file_name,delimiter='\t')
        df_liar_mapping = df_liar.iloc[:,[1,2]]
        df_liar_mapping.columns=['IsFakeNews','text']
        df_liar_mapping['label'] = df_liar_mapping.IsFakeNews.apply(lambda x: 0 if x=='false' else 1)
        df_liar_mapping.drop('IsFakeNews',axis=1,inplace=True)
        dfs.append(df_liar_mapping)
    df_test = pd.concat([dfs[1],dfs[2]],axis=0)
    return (dfs[0],df_test)
df_train_liar_dataset, df_test_liar_datasset = GetMappingLiarDataset(['Liar_Dataset/train.tsv','Liar_Dataset/test.tsv','Liar_Dataset/valid.tsv'])

/tmp/ipykernel_36078/4075386623.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_liar_mapping['label'] = df_liar_mapping.IsFakeNews.apply(lambda x: 0 if x=='false' else 1)
/tmp/ipykernel_36078/4075386623.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_liar_mapping.drop('IsFakeNews',axis=1,inplace=True)
/tmp/ipykernel_36078/4075386623.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [8]:
df_fake_misinfo = pd.read_csv('Misinfo/DataSet_Misinfo_FAKE.csv')
df_fake_misinfo['label'] = df_fake_misinfo['text'].apply(lambda x:1)
df_fake_misinfo = df_fake_misinfo.loc[:,['text','label']]
df_true_misinfo = pd.read_csv('Misinfo/DataSet_Misinfo_TRUE.csv')
df_true_misinfo['label'] = df_true_misinfo['text'].apply(lambda x:0)
df_true_misinfo = df_true_misinfo.loc[:,['text','label']]
df_true_misinfo.dropna(inplace=True)
df_misinfo = pd.concat([df_fake_misinfo,df_true_misinfo])

In [9]:
df_fake = pd.read_csv('Fake_Real_News_Kaggle/Fake.csv')
df_fake = pd.DataFrame(df_fake.text,columns=['text'])
df_fake['label'] = df_fake.text.apply(lambda x : 1)
df_real = pd.read_csv('Fake_Real_News_Kaggle/True.csv')
df_real = pd.DataFrame(df_real.text)
df_real['label'] = df_real.text.apply(lambda x : 0)
df_real_news_kaggle = pd.concat([df_fake,df_real],axis=0)

In [10]:
df_welfake = pd.read_csv('Wel_Fake_Kaggle/WELFake_Dataset.csv')
df_welfake = df_welfake.loc[:,['text','label']]
df_welfake.dropna(inplace=True)

In [11]:
df_train_input = pd.read_csv('Input_Fake_News/train.csv',delimiter=';')
df_train_input = df_train_input.loc[:,['text','label']]
df_test_input = pd.read_csv('Input_Fake_News/test.csv',delimiter=';')
df_test_input = df_test_input.loc[:,['text','label']]
df_evaluation_input = pd.read_csv('Input_Fake_News/evaluation.csv',delimiter=';')
df_evaluation_input = df_evaluation_input.loc[:,['text','label']]

In [12]:
df_final_train = pd.concat([df_train_fake_news_kaggle, df_train_liar_dataset, df_test_liar_datasset, df_misinfo, df_real_news_kaggle, df_welfake, df_train_input, df_test_input])


In [13]:
import numpy as np 


In [14]:
df_fake = df_final_train[df_final_train['label'] == 1]
df_true = df_final_train[df_final_train['label'] == 0]
np.random.seed(42)
indexes_fake = np.random.randint(0,df_fake.shape[0],12500)
indexes_true = np.random.randint(0,df_true.shape[0],10000)
df_fake = df_fake.iloc[indexes_fake]
df_true = df_true.iloc[indexes_true]


In [15]:
df = pd.concat([df_fake,df_true])

BaseLine = 55%

In [16]:
from nltk.corpus import words
english_words = set(words.words())


In [17]:
english_words = set(words.words())

In [18]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [37]:
def RemoveNonAlpha(text):
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    test = False
    for char in text:
        if char != ' ' and char not in alphabet:
            test = True
    if test == True:
        return ' '
    return text

In [53]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    output = np.asarray(pos_tag)
    for i in range(len(pos_tag)):
        if pos_tag[i][1].startswith('J'):
            output[i][1] = wordnet.ADJ
        elif pos_tag[i][1].startswith('V'):
            output[i][1] = wordnet.VERB
        elif pos_tag[i][1].startswith('R'):
            output[i][1] = wordnet.ADV
        else:
            output[i][1] = wordnet.NOUN
    return output

def Tokenize(text):
    text = text.lower()
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    token_liste = [token for token in word_tokenize(text) if token.isalpha() and token not in stop_words]
    test = False
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    for mot in token_liste:
        for char in mot:
            if char not in alphabet:
                test = True
    if test == False:
        final_liste = [token for token in token_liste]
        tags = nltk.pos_tag(final_liste)
        wordnet_input = get_wordnet_pos(tags)
        lem_tokens = [lemmatizer.lemmatize(t,tag) for t,tag in wordnet_input]
        return lem_tokens
    return [' ']

In [55]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/cyrius8360/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [56]:
df['tokenize'] = df['text'].apply(Tokenize)

In [58]:
df['tokenized_text'] = df['tokenize'].apply(lambda tokens:' '.join(tokens).lower())


In [59]:
df = df[df['tokenized_text'] != ' ']

In [60]:
df_cleaned = df.loc[:,['tokenized_text','label']]
df_cleaned

,tokenized_text,label
66453,mint press news tue oct utc inmate make sandwi...,1
13013,kabul reuters islamic state claim responsibili...,1
30750,donald trump describe many sound like comic bo...,1
62491,rmuse sit oct pm take conspiracy theory obstru...,1
43627,trey gowdy chairman benghazi select committee ...,1
...,...,...
33297,amsterdam reuters least two people kill severa...,0
34518,asuncion reuters paraguayan senator mario abdo...,0
36870,reuters canadian prime minister justin trudeau...,0
3598,since obamacare become law new job job,0


In [51]:
X = df_cleaned['tokenized_text']
y = df_cleaned['label']

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_cleaned = vectorizer.fit_transform(X)
print(vectorizer.get_feature_names_out())

['aa' 'aaa' 'aaahhh' ... 'zytsov' 'zyuganov' 'zzzzzzzz']


In [62]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_cleaned,y,test_size=0.2)

In [63]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
lreg = LogisticRegression()
lreg.fit(X_train, y_train)
y_pred = lreg.predict(X_test)
accuracy_score(y_test, y_pred), f1_score(y_test, y_pred)

(0.742455655378945, 0.7814698983580923)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'solver': ['lbfgs', 'liblinear', 'saga', 'newton-cg', 'sag'],
    'max_iter': [100000]
}

logreg = LogisticRegression()
grid_search = GridSearchCV(logreg, param_grid, cv=5)
grid_search.fit(X_train, y_train)

In [142]:
# from sklearn.svm import SVC
# svc = SVC()
# svc.fit(X_train, y_train)
# y_pred = svc.predict(X_test)
# accuracy_score(y_test, y_pred), f1_score(y_test, y_pred)

(0.7373333333333333, 0.7711076684740512)

In [143]:
# from sklearn.neighbors import KNeighborsClassifier
# knn = KNeighborsClassifier(n_neighbors=3)
# knn.fit(X_train, y_train)
# y_pred_knn = knn.predict(X_test)
# accuracy_score(y_test, y_pred_knn), f1_score(y_test, y_pred_knn)

(0.5842222222222222, 0.7181804488627807)